<a href="https://colab.research.google.com/github/Ry02024/SIGNATE/blob/main/%E3%82%A2%E3%83%A4%E3%83%A1%E3%81%AE%E5%88%86%E9%A1%9E/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9%E3%82%A2%E3%83%97%E3%83%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#streamlit：簡単な分析と機械学習

##環境構築

In [2]:
!pip install streamlit -q
!pip install pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00


In [3]:
import streamlit as st
from pyngrok import ngrok

In [4]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [5]:
from pyngrok import ngrok
import os

# ngrokトークンを設定
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

##実行ファイル

###version1

In [6]:
!pip install japanize-matplotlib -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [49]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# --- 関数 ---
def load_data(file, file_type="訓練"):
    """
    ファイルをアップロードし、データフレームを読み込む。
    """
    if file is not None:
        data = pd.read_csv(file, delimiter='\t')
        if 'id' in data.columns:
            data.drop(columns='id', inplace=True)
        st.write(f"{file_type}データの最初の5行:")
        st.write(data.head())
        return data
    return None

def visualize_data_analysis(data):
    """
    データ分析の可視化を実行する。
    """
    if data is not None:
        st.write('基本統計量:')
        st.write(data.describe())

        st.write('特徴量のヒストグラム:')
        fig, ax = plt.subplots(figsize=(15, 10))
        data.hist(bins=15, ax=ax)
        st.pyplot(fig)

        st.write('散布図とカーネル密度推定:')
        pair_plot_fig = sns.pairplot(data, hue='class', markers=["o", "s", "D"], palette="bright")
        st.pyplot(pair_plot_fig)

        st.write('相関行列:')
        numeric_data = data.select_dtypes(include=['number'])
        fig, ax = plt.subplots()
        sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', ax=ax)
        st.pyplot(fig)

        st.session_state['visualization_shown'] = True  # 可視化が実行されたことを記録

def train_and_evaluate_model(data):
    """
    モデルを訓練し、評価指標を表示する。
    """
    global best_model, scaler # best_modelとscalerをグローバル変数として宣言
    if data is not None:
        X_train = data.drop('class', axis=1)
        y_train = data['class']
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)

        model = RandomForestClassifier(random_state=42)
        param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30]}
        grid_search = GridSearchCV(model, param_grid, cv=5)
        grid_search.fit(X_train_scaled, y_train)
        best_model = grid_search.best_estimator_

        y_train_pred = best_model.predict(X_train_scaled)
        st.write("訓練データの精度: ", accuracy_score(y_train, y_train_pred))
        st.write("適合率:", precision_score(y_train, y_train_pred, average='macro'))
        st.write("再現率:", recall_score(y_train, y_train_pred, average='macro'))
        st.write("F1スコア:", f1_score(y_train, y_train_pred, average='macro'))

        fig, ax = plt.subplots()
        sns.heatmap(confusion_matrix(y_train, y_train_pred), annot=True, fmt='d', cmap='Blues', ax=ax)
        plt.title("訓練データの混同行列")
        st.pyplot(fig)

def predict_test_data(data):
    """
    テストデータの予測を実行する。
    """
    global best_model, scaler
    if data is not None and best_model is not None:
        X_test_scaled = scaler.transform(data)
        y_test_pred = best_model.predict(X_test_scaled)
        st.write("テストデータの予測結果:")
        st.write(pd.DataFrame({'Prediction': y_test_pred}))

def update_model_with_additional_data(additional_data):
    """
    追加データでモデルを更新する。
    """
    global train_data, best_model
    if additional_data is not None and train_data is not None:
        train_data = pd.concat([train_data, additional_data], ignore_index=True)
        st.write("統合後のデータの最初の5行:")
        st.write(train_data.head())
        if best_model is not None:
            train_and_evaluate_model(train_data)

# --- main ---
def main():
    st.title('アヤメの品種分類アプリ')
    st.write('このアプリはアヤメのデータセットを分析し、品種を予測します。')

    # データフレームとモデルを格納する変数
    global train_data, test_data, best_model, scaler

    # ファイルアップローダー
    train_file = st.file_uploader("訓練データをアップロードしてください（train.tsv）", type=['tsv'])
    test_file = st.file_uploader("テストデータをアップロードしてください（test.tsv）", type=['tsv'])

    train_data = load_data(train_file)
    test_data = load_data(test_file, "テスト")

    # セッションステートの初期化
    if 'visualization_shown' not in st.session_state:
        st.session_state['visualization_shown'] = False

    # 可視化の実行
    visualize = st.checkbox('データを可視化して分析')
    if visualize and train_data is not None:
        visualize_data_analysis(train_data)

    # モデル訓練ボタン
    train_model = st.checkbox('モデルを訓練して評価')
    if train_model and train_data is not None:
        train_and_evaluate_model(train_data)

    # 予測ボタン
    predict = st.checkbox('テストデータを予測')
    if predict and test_data is not None:
        predict_test_data(test_data)

    # 追加データのアップロード
    additional_data = load_data(st.file_uploader("追加のデータセットをアップロードして訓練データに追加", type=['tsv']), "追加")
    if additional_data is not None:
        update_model_with_additional_data(additional_data)

# アプリケーションの実行
if __name__ == "__main__":
    main()


Overwriting streamlit_app.py


##アプリの起動

In [50]:
# ngrokを介してStreamlitを公開
public_url = ngrok.connect(addr='8501')
print('Public URL:', public_url)

Public URL: NgrokTunnel: "https://bb24-34-136-117-121.ngrok-free.app" -> "http://localhost:8501"


In [51]:
# Streamlitアプリケーションの起動
!streamlit run streamlit_app.py >/dev/null

/content/streamlit_app.py:35: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared.
  data.hist(bins=15, ax=ax)
/content/streamlit_app.py:35: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared.
  data.hist(bins=15, ax=ax)
/content/streamlit_app.py:35: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared.
  data.hist(bins=15, ax=ax)
/content/streamlit_app.py:35: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared.
  data.hist(bins=15, ax=ax)
